# 🛠 Setup

In [ ]:
%cd /content

In [ ]:
!git clone -b automation-script https://github.com/truefoundry/truefoundry-examples.git

In [ ]:
pip install -U "servicefoundry==0.6.6"

In [ ]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [ ]:
!sfy login

# ⚡Training Model

In [ ]:
# Copy Workspace FQN from https://app.truefoundry.com/workspaces
# Copy API Key from https://app.truefoundry.com/settings
from getpass import getpass

WORKSPACE = input("Please enter your Workspace FQN, (https://app.truefoundry.com/workspaces):-")
TFY_API_KEY = getpass("Please enter your API Key, (https://app.truefoundry.com/settings):-")

In [ ]:
%cd /content/truefoundry-examples/end-to-end-examples/red-wine-classification/train

In [7]:
!pygmentize train.py

import os
import time
import mlfoundry
from datetime import datetime
from train_model import train_model
from sklearn.metrics import accuracy_score, f1_score
from dataset import get_initial_data

s = time.time()

# You can bring data from your own sources
X_train, X_test, y_train, y_test = get_initial_data(test_size=0.1, random_state=42)

model, metadata = train_model(X_train, y_train, X_test, y_test)
e = time.time()

print(f"################### Time taken to log models {e-s} ")

features = [{"name": column, "type": "float"} for column in X_train.columns]
schema = {"features": features, "prediction": "categorical"}
print(f"Schema: {schema}")

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# logging the data for experiment tracking
# You can push the model to your choice of storage or model registry.
run = mlfoundry.get_client().create_run(project_name="red-wine-quality-demo", run_name=f"train-{datetime.now().strftime('%m-%d-%Y')}")
run.log_params(model.get_p

In [ ]:
import argparse
import logging

from servicefoundry import Build, Job, PythonBuild, Resources

logging.basicConfig(level=logging.INFO)

# servicefoundry uses this specification to automatically create a Dockerfile and build an image,
python_build = PythonBuild(
    python_version="3.9",
    command="python train.py",
)
env = {
    "TFY_API_KEY": TFY_API_KEY,
}
job = Job(
    name="red-wine-train",
    image=Build(build_spec=python_build),
    env=env,
    resources=Resources(
        cpu_request=1, cpu_limit=1.5, memory_request=1000, memory_limit=1500
    ),
)
output_train = job.deploy(workspace_fqn=WORKSPACE)

In [ ]:
# In order to trigger the above job, go to the Dashboard --> Deployments --> red-wine-train job --> 'Run Job'   OR   Run below script to trigger a job

from servicefoundry.internal.experimental import trigger_job

# Job Deployment FQN can be found at Dashboard --> Deployments --> red-wine-train job --> FQN
JOB_DEPLOYMENT_FQN = output_train.fqn

trigger_job(deployment_fqn=JOB_DEPLOYMENT_FQN, command="python train.py")

# ☁ Deploying FastAPI Prediction Service

In [ ]:
# Copy the FQN of the latest version of the model from Experiments Tracking tab of TrueFoundry --> project name (red-wine-quality-demo by default) --> Models --> FQN
MODEL_VERSION_FQN = input("Please enter your Model Version FQN:-")

In [ ]:
%cd /content/truefoundry-examples/end-to-end-examples/red-wine-classification/infer_realtime

In [9]:
!pygmentize infer_realtime.py

import os
import time
import uuid
from typing import List

import mlfoundry as mlf
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel, Field

# fetching the model object from the model vesion fqn
MODEL_VERSION_FQN = os.getenv("MLF_MODEL_VERSION_FQN")
client = mlf.get_client()
model_version = client.get_model(MODEL_VERSION_FQN)
model = model_version.load()

app = FastAPI(root_path=os.getenv("TFY_SERVICE_ROOT_PATH"), docs_url="/")

# creating the inference request object
class WinePredictionRequest(BaseModel):
    data_id: str = Field(default_factory=lambda: uuid.uuid4().hex)
    fixed_acidity: float
    volatile_acidity: float
    citric_acid: float
    residual_sugar: float
    chlorides: float
    free_sulfur_dioxide: float
    total_sulfur_dioxide: float
    density: float
    pH: float
    sulphates: float
    alcohol: float

    class Config:
        alias_generator = lambda string: string.replace("_", " ")
        allow_population_by_field_name = True



In [ ]:
import argparse
import logging

from servicefoundry import Build, PythonBuild, Resources, Service

logging.basicConfig(level=logging.INFO)

# creating a service object and defining all the configurations
service = Service(
    name="red-wine-fastapi",
    image=Build(
        build_spec=PythonBuild(
            command="uvicorn infer_realtime:app --port 4000 --host 0.0.0.0",
            python_version="3.9",
        ),
    ),
    env={
        "TFY_API_KEY": TFY_API_KEY,
        "MLF_MODEL_VERSION_FQN": MODEL_VERSION_FQN,
    },
    ports=[{"port": 4000}],
    resources=Resources(
        cpu_request=0.5, cpu_limit=1.5, memory_limit=2500, memory_request=1500
    ),
)
output_fastapi = service.deploy(workspace_fqn=WORKSPACE)

In [ ]:
# Set this as the FastAPI Endpoint of the deployed service
INFERENCE_SERVER_URL = input("Enter the FastAPI Endpoint of the deployed service:-")

Click on the dashboard link in the terminal to open the service deployment page with FastAPI Endpoint

# 👨‍💻 Testing Deployed Model

In [ ]:
import requests
from urllib.parse import urljoin

features_list = [
    {
        "data id": "string", "fixed acidity": 7.5, "volatile acidity": 0.42,
        "citric acid": 0.32, "residual sugar": 2.7, "chlorides": 0.067,
        "free sulfur dioxide": 7.0, "total sulfur dioxide": 25.0, 
        "density": 0.99628, "pH": 3.24, "sulphates": 0.44, "alcohol": 10.4
    }
]

data = requests.post(url=urljoin(INFERENCE_SERVER_URL, './predict'), json=features_list).json()
print(data)

# ⚡Running Batch Inference

In [ ]:
%cd /content/truefoundry-examples/end-to-end-examples/red-wine-classification/infer_batch

In [11]:
!pygmentize infer_batch.py

import os
import random
import time
from urllib.parse import urljoin

import mlfoundry as mlf
import pandas as pd
import requests

client = mlf.get_client()
request_url = os.getenv("INFERENCE_SERVER_URL")

DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"


# generate random input samples from the original dataset
def get_input_data(num_samples=random.randint(15, 30)):
    df = pd.read_csv(DATASET_URL, sep=";")
    df = df.sample(n=num_samples)
    y = df.pop("quality").to_list()
    X = df.to_dict("records")
    X = [{key.replace(" ", "_"): value for key, value in row.items()} for row in X]
    print(f"Fetched {num_samples} random data points")
    return X, y


features_list, actuals_list = get_input_data()

print("Running predictions...")
# finding predictions from the inference server
predictions_list = requests.post(
    url=urljoin(request_url, "./predict"), json=features_list
).json()

print("Logging predictions...")
# log

In [ ]:
import argparse
import logging

from servicefoundry import Build, Job, PythonBuild, Schedule

logging.basicConfig(level=logging.INFO)

job = Job(
    name="red-wine-batch",
    image=Build(
        build_spec=PythonBuild(command="python infer_batch.py"),
    ),
    env={
        "INFERENCE_SERVER_URL": INFERENCE_SERVER_URL,
        "TFY_API_KEY": TFY_API_KEY,
    },
    trigger=Schedule(schedule="*/10 * * * *"),
)
job.deploy(workspace_fqn=WORKSPACE)

# ☁ Deploying Streamlit Demo

In [ ]:
%cd /content/truefoundry-examples/end-to-end-examples/red-wine-classification/demo

In [13]:
!pygmentize demo.py

import os
from urllib.parse import urljoin

import requests
import streamlit as st

def fetch(session, url):
    try:
        result = session.get(url)
        return result.json()
    except Exception:
        return {}


mini = [4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000]
maxi = [15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000]



def main():
    # don't forget the trailing slash
    request_url = os.getenv("INFERENCE_SERVER_URL")

    st.set_page_config(page_title="Trial", page_icon="🤖")
    st.title("Wine Quality Prediction")
    with st.form("my_form"):        
        fixed_acidity = st.slider("Fixed Acidity",min_value=0.0,max_value=16.0,step=0.1,key="fixed_acidity")
        volatile_acidity = st.slider("Volatile Acidity",min_value=0.0,max_value=1.6,step=0.01,key="volatile_acidity")
        citric_acid = st.slider("Citric Acid",min_value=mini[2],max_value=maxi[2],step=

In [ ]:
import argparse
import logging

from servicefoundry import Build, Job, PythonBuild, Schedule

logging.basicConfig(level=logging.INFO)

# creating a service object and defining all the configurations
service = Service(
    name="red-wine-demo",
    image=Build(
        build_spec=PythonBuild(
            command="streamlit run demo.py",
            python_version="3.9",
        ),
    ),
    env={
        "INFERENCE_SERVER_URL": INFERENCE_SERVER_URL,
        "TFY_API_KEY": TFY_API_KEY,
    },
    ports=[{"port": 8501}], #In public cloud deployment TrueFoundry exposes port 8501
    resources=Resources(
        cpu_request=0.5, cpu_limit=0.5, memory_limit=2500, memory_request=1500
    ),
)
service.deploy(workspace_fqn=WORKSPACE)

Click on the dashboard link in the terminal. Click on the "Endpoint" link on the dashboard to open the streamlit demo